In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import( ChatPromptTemplate,
                              HumanMessagePromptTemplate,
                              MessagesPlaceholder)

from langchain.agents import OpenAIFunctionsAgent, AgentExecutor
from tools.sql import run_query_tool

In [13]:
from dotenv import load_dotenv
load_dotenv()

True

In [14]:
chat= ChatOpenAI()
prompt= ChatPromptTemplate(
    messages= [HumanMessagePromptTemplate.from_template("{input}"),
               MessagesPlaceholder(variable_name="agent_scratchpad")]
)

In [15]:
tools= [run_query_tool]
agent= OpenAIFunctionsAgent(llm= chat,
                            prompt= prompt,
                            tools= tools)
agent_executor= AgentExecutor(agent= agent,
                              verbose= True,
                              tools= tools)

In [1]:
# agent_executor("how many users are in the database?")
# print("agent")

In [5]:
from langchain.chat_models import ChatOpenAI
llm= ChatOpenAI(model= "gpt-3.5-turbo")

## OPenAI functions

In [7]:
import json

def get_current_weather(loc, unit= "celsius"):
    weather_info = {
        "loc" : loc,
        "temperature" : 60,
        "unit" : unit,
        "forcst" : ["sunny", "windy"]
    }
    return json.dumps(weather_info)

In [10]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [8]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [12]:
import openai

response = openai.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)

In [32]:
print(response)

ChatCompletion(id='chatcmpl-8h03Ruwfm6xCZ74GXaqjqtDq1mW4U', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None))], created=1705259005, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=18, prompt_tokens=82, total_tokens=100))


In [30]:
response_message = response.choices[0].message
json.loads(response_message.function_call.arguments)
response_message


ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None)

In [29]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

In [37]:
from langchain.agents import create_openai_functions_agent
from tools.sql import run_query_tool, list_tables, describe_tables_tool
from langchain.prompts import( ChatPromptTemplate,
                              HumanMessagePromptTemplate,
                              MessagesPlaceholder)
from langchain.schema import SystemMessage
from langchain.agents import AgentExecutor

tables= list_tables()

prompt= ChatPromptTemplate(
    messages= [
        SystemMessage(content= ("You are an AI that has access to a SQLite database.\n"
        f"The database has tables of:{tables}\n"
        "Do not make any assumptions about what tables exist or what columns exist. Instead, use the 'describe_tables' function" ) ),
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")]
)
tools= [run_query_tool, describe_tables_tool]

agent = create_openai_functions_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)



In [39]:
agent_executor.invoke({"input": "How many users have provided a shipping address?"})

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "How many users have provided a shipping address?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "How many users have provided a shipping address?",
  "intermediate_steps": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": "How many users have provided a shipping address?",
  "intermediate_steps": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": "How many users have provided a shipping address?",
  "intermediate_steps": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad> > 5:chain:Ru

{'input': 'How many users have provided a shipping address?',
 'output': 'There are 2000 users who have provided a shipping address.'}

In [40]:
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

In [41]:
@tool
def search(query: str) -> str:
    """Look up things online"""
    return "LangChain"

print(search.name)
print(search.description)
print(search.args)

search
search(query: str) -> str - Look up things online
{'query': {'title': 'Query', 'type': 'string'}}


In [43]:
@tool
def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a * b

print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
multiply(a: int, b: int) -> int - Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [45]:
class SearchInput(BaseModel):
    query: str= Field(description= "should be a search query.")

@tool ("search-tool", args_schema= SearchInput, return_direct= True)
def search(query: str) -> str:
    """Look up things online"""
    return "LangChain"

print(search.args)

{'query': {'title': 'Query', 'description': 'should be a search query.', 'type': 'string'}}
